In [62]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [ ]:
#2015-summary.json

In [26]:
df = spark.read.format('json').load("/home/jovyan/work/start_spark/start_spark/running_spark_data/2015-summary.json")

In [25]:
#df.select('DEST_COUNTRY_NAME').distinct()

In [28]:
df.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [29]:
df1 = df.select("DEST_COUNTRY_NAME").distinct().cache()
df1.count()

132

In [31]:
#row calss를 이용한 단일 레코드 생성
from pyspark.sql import Row
myRow = Row('hello', None, 1, False)
myRow

<Row('hello', None, 1, False)>

In [38]:
#새로운 컬럼 추가하기
from pyspark.sql.functions import expr
df3 =df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME'))

In [ ]:
#withinCountry가 true인 아이들 

In [44]:
df3.filter("withinCountry == true").show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [45]:
df3.filter(expr("withinCountry == true")).show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [ ]:
#case when 카운트 10 이하 under 10 이상 upper로 변환 

In [53]:
df4 = df.withColumn('category', expr("CASE WHEN count < 10 THEN 'under' WHEN count > 10 THEN 'upper' END"))
df4.show()

+--------------------+-------------------+-----+--------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|
+--------------------+-------------------+-----+--------+
|       United States|            Romania|   15|   upper|
|       United States|            Croatia|    1|   under|
|       United States|            Ireland|  344|   upper|
|               Egypt|      United States|   15|   upper|
|       United States|              India|   62|   upper|
|       United States|          Singapore|    1|   under|
|       United States|            Grenada|   62|   upper|
|          Costa Rica|      United States|  588|   upper|
|             Senegal|      United States|   40|   upper|
|             Moldova|      United States|    1|   under|
|       United States|       Sint Maarten|  325|   upper|
|       United States|   Marshall Islands|   39|   upper|
|              Guyana|      United States|   64|   upper|
|               Malta|      United States|    1|   under|
|            A

In [60]:
spark.stop()

In [72]:
emp_df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load("/home/jovyan/work/start_spark/start_spark/running_spark_data/emp.csv")

In [74]:
dept_df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load("/home/jovyan/work/start_spark/start_spark/running_spark_data/dept.csv")

In [ ]:
#집계함수

In [ ]:
#emp_df, dept_df, 

In [76]:
dept_df.take(3)

[Row(deptno=10, dname='ACCOUNTING', loc='NEW YORK'),
 Row(deptno=20, dname='RESEARCH', loc='DALLAS'),
 Row(deptno=30, dname='SALES', loc='CHICAGO')]

In [77]:
emp_df.take(3)

[Row(empno=7369, ename='SMITH', job='CLERK', mgr=7902, hiredate=datetime.date(1980, 12, 17), sal=800, comm=None, deptno=20),
 Row(empno=7499, ename='ALLEN', job='SALESMAN', mgr=7698, hiredate=datetime.date(1981, 2, 20), sal=1600, comm=300, deptno=30),
 Row(empno=7521, ename='WARD', job='SALESMAN', mgr=7698, hiredate=datetime.date(1981, 2, 22), sal=1250, comm=500, deptno=30)]

In [78]:
emp_df.select('ENAME','DEPTNO').show()

+------+------+
| ENAME|DEPTNO|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
| BLAKE|    30|
| CLARK|    10|
| SCOTT|    20|
|  KING|    10|
|TURNER|    30|
| ADAMS|    20|
| JAMES|    30|
|  FORD|    20|
|MILLER|    10|
|  JACK|    70|
+------+------+



In [79]:
emp_df.select('*').where('deptno=20').show()

+-----+-----+-------+----+----------+----+----+------+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+-------+----+----------+----+----+------+
| 7369|SMITH|  CLERK|7902|1980-12-17| 800|NULL|    20|
| 7566|JONES|MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7876|ADAMS|  CLERK|7788|1987-05-23|1100|NULL|    20|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|
+-----+-----+-------+----+----------+----+----+------+



In [82]:
emp_df.selectExpr('count(*)').show()

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [87]:
from pyspark.sql.functions import countDistinct
emp_df.select(countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [91]:
from pyspark.sql.functions import approx_count_distinct
emp_df.select(approx_count_distinct('job', 0.1)).show()

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [ ]:
#first # last # min #max #sum #avg sql 문장 X  function으로 처리

In [292]:
from pyspark.sql.functions import first, last, min, max, sum, avg,mean, col, count, Column, round, std, desc, rank, when

In [110]:
emp_df.select(min('sal')).show()

+--------+
|min(sal)|
+--------+
|     800|
+--------+



In [111]:
emp_df.select(first('sal')).show()

+----------+
|first(sal)|
+----------+
|       800|
+----------+



In [112]:
emp_df.select(last('sal')).show()

+---------+
|last(sal)|
+---------+
|     3200|
+---------+



In [115]:
emp_df.select(avg('sal')).show()

+------------------+
|          avg(sal)|
+------------------+
|2148.3333333333335|
+------------------+



In [118]:
emp_df.select(sum('sal')).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [131]:
emp_df.select(sum('sal')).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [132]:
emp_df.select(mean('sal')).show()

+------------------+
|          avg(sal)|
+------------------+
|2148.3333333333335|
+------------------+



In [126]:
emp_df.select('sal').where('deptno=20').show()

+----+
| sal|
+----+
| 800|
|2975|
|3000|
|1100|
|3000|
+----+



In [128]:
emp_df.select(sum('sal').where('deptno=20').show()

+----+
| sal|
+----+
| 800|
|2975|
|3000|
|1100|
|3000|
+----+



In [137]:
emp_df.select(sum(col('sal'))).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [138]:
emp_df.where('deptno=20').select(sum('sal')).show()

+--------+
|sum(sal)|
+--------+
|   10875|
+--------+



In [ ]:
total_salary /total_transaction. avg_salary, mean_salary

In [148]:
emp_df.where('deptno=20').selectExpr('sum(distinct sal)').show()

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|             7875|
+-----------------+



In [169]:
emp_df.select(sum('sal').alias('total_salary'),  avg('sal').alias('avg_salary'), mean('sal').alias('mean_salary'), count('*').alias('total_trainsection')).show()

+------------+------------------+------------------+------------------+
|total_salary|        avg_salary|       mean_salary|total_trainsection|
+------------+------------------+------------------+------------------+
|       32225|2148.3333333333335|2148.3333333333335|                15|
+------------+------------------+------------------+------------------+



In [168]:
emp_df.select(count('*').alias('trainsection')).show()

+------------+
|trainsection|
+------------+
|          15|
+------------+



In [180]:
emp_df.select(sum('sal').alias('total_salary'),  avg('sal').alias('avg_salary'), mean('sal').alias('mean_salary'), count('*').alias('total_transaction')).selectExpr('total_salary/total_transaction', 'avg_salary', 'mean_salary').show()

+----------------------------------+------------------+------------------+
|(total_salary / total_transaction)|        avg_salary|       mean_salary|
+----------------------------------+------------------+------------------+
|                2148.3333333333335|2148.3333333333335|2148.3333333333335|
+----------------------------------+------------------+------------------+



In [183]:
emp_df.select(
    sum('sal').alias('total_salary'),
    round(avg('sal'), 2).alias('avg_salary'),  # avg() 결과에 round 적용
    count('*').alias('total_transaction'),
    round(sum('sal') / count('*'), 2).alias('calculated_avg_salary') # 계산 결과에 round 적용
).show()

+------------+----------+-----------------+---------------------+
|total_salary|avg_salary|total_transaction|calculated_avg_salary|
+------------+----------+-----------------+---------------------+
|       32225|   2148.33|               15|              2148.33|
+------------+----------+-----------------+---------------------+



In [186]:
#그룹화 
emp_df.groupBy('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [ ]:
#select job,
#    count(job),
#    sum(sal)
#group by job

In [208]:
group_df=emp_df.groupby('job').agg(
    count('job').alias('qty'),
    expr('count(job)'),
    sum('sal'))

In [209]:
group_df.show()

+---------+---+----------+--------+
|      job|qty|count(job)|sum(sal)|
+---------+---+----------+--------+
|  ANALYST|  2|         2|    6000|
| SALESMAN|  4|         4|    5600|
|    CLERK|  5|         5|    7350|
|  MANAGER|  3|         3|    8275|
|PRESIDENT|  1|         1|    5000|
+---------+---+----------+--------+



In [222]:
#sal평균 sal_avg, 표준편차 sal_stdev를 job별로 계산해서 출력 소수점 두자리까지
group_df=emp_df.groupby('job').agg(
    count('job').alias('qty'),
    expr('count(job)'),
    sum('sal'),round(std('sal'), 2).alias('stdev'), round(avg('sal'), 2).alias('sal_avg'))


In [225]:
#sal평균 sal_avg, 표준편차 sal_stdev를 job별로 계산해서 출력 소수점 두자리까지
group_df=emp_df.groupby('job').agg(
    count('job').alias('qty'),
    sum('sal'),round(std('sal'), 2).alias('stdev'), round(avg('sal'), 2).alias('sal_avg'))

In [226]:
group_df.show()

+---------+---+--------+------+-------+
|      job|qty|sum(sal)| stdev|sal_avg|
+---------+---+--------+------+-------+
|  ANALYST|  2|    6000|   0.0| 3000.0|
| SALESMAN|  4|    5600|177.95| 1400.0|
|    CLERK|  5|    7350|984.63| 1470.0|
|  MANAGER|  3|    8275|274.24|2758.33|
|PRESIDENT|  1|    5000|  NULL| 5000.0|
+---------+---+--------+------+-------+



In [227]:
group_df=emp_df.groupby('job').agg(
    count('job').alias('qty'),
    expr('count(job)'),
    sum('sal'),std('sal').alias('stdev')).limit(10).show()

+---------+---+----------+--------+------------------+
|      job|qty|count(job)|sum(sal)|             stdev|
+---------+---+----------+--------+------------------+
|  ANALYST|  2|         2|    6000|               0.0|
| SALESMAN|  4|         4|    5600|177.95130420052183|
|    CLERK|  5|         5|    7350| 984.6319109189992|
|  MANAGER|  3|         3|    8275|274.24137786507225|
|PRESIDENT|  1|         1|    5000|              NULL|
+---------+---+----------+--------+------------------+



In [230]:
emp_df.orderBy(desc('sal')).limit(10).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
+-----+------+---------+----+----------+----+----+------+



In [242]:
#윈도우함수
from pyspark.sql.window import Window
windowspec=Window.orderBy(desc('sal'))
salAllRank=rank().over(windowspec)

In [244]:
emp_df.withColumn('salary_rank', salAllRank).show(10)

+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|         10|
+-----+------+---------+----+----------+----+----+------+-----------+
only showing top 10 

In [ ]:
#직무별로 rank
#window partitionby() (PARTITION BY job
#job_rank_df 작성


In [245]:
#윈도우함수
from pyspark.sql.window import Window
windowspec=Window.orderBy(desc('sal'))
salAllRank=rank().over(windowspec)

In [246]:
emp_df.dtypes

[('empno', 'int'),
 ('ename', 'string'),
 ('job', 'string'),
 ('mgr', 'int'),
 ('hiredate', 'date'),
 ('sal', 'int'),
 ('comm', 'int'),
 ('deptno', 'int')]

In [267]:
emp_df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: date (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [253]:
from pyspark.sql import Window
from pyspark.sql.functions import desc, rank


window_spec = Window.partitionBy('job').orderBy(desc('sal'))

ranked_df = emp_df.withColumn(
    "job_rank",
    rank().over(window_spec)
)

ranked_df.show()

+-----+------+---------+----+----------+----+----+------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|job_rank|
+-----+------+---------+----+----------+----+----+------+--------+
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|       1|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|       1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|       1|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|       2|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|       3|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|       4|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|       5|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|       1|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|       2|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|       3|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|       1|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|     

In [271]:
    windowspec1 = Window.partitionBy('job').orderBy(desc('sal'))
    salAllRank = rank().over(windowspec1)

In [273]:
# 2. 윈도우 스펙을 적용하여 각 부서별 급여 순위
emp_df_rank = emp_df.withColumn(
    'dept_rank', 
    salAllRank
    #rank().over(windowspec1)
)
emp_df_rank.show()

+-----+------+---------+----+----------+----+----+------+---------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|dept_rank|
+-----+------+---------+----+----------+----+----+------+---------+
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|        1|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|        1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|        1|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|        2|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|        3|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|        4|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|        5|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|        1|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|        2|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|        3|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|        1|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 30

In [255]:
#부서별 누적급여 sum('sal).over()

In [280]:

windowspec1 = Window.partitionBy('deptno').orderBy(desc('sal'))
# 2. 윈도우 스펙을 적용하여 각 부서별 'sal'의 누적 합계를 계산합니다.
# rank() 대신 sum('sal')을 사용합니다.
cumulative_sal_by_dept = emp_df.withColumn(
    'cumulative_sal', 
    sum('sal').over(windowspec1)
)

# 결과를 출력하여 확인합니다.
# 각 부서(deptno) 내에서 급여가 높은 직원부터 누적된 급여가 표시됩니다.
cumulative_sal_by_dept.show()

+-----+------+---------+----+----------+----+----+------+--------------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|cumulative_sal|
+-----+------+---------+----+----------+----+----+------+--------------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          5000|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|          7450|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|          8750|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          6000|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|          6000|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|          8975|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|         10075|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|         10875|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|          2850|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          4450|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0| 

In [281]:
#부서별 평균급여와  직원 개별 급여 비교

In [288]:
windowspec1 = Window.partitionBy('deptno').orderBy(desc('sal'))

avg_sal_dept = emp_df.withColumn(
    'avg_sal', 
    round(avg('sal').over(windowspec1),2)
)

avg_sal_dept.show()

+-----+------+---------+----+----------+----+----+------+-------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|avg_sal|
+-----+------+---------+----+----------+----+----+------+-------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10| 5000.0|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10| 3725.0|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|2916.67|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20| 3000.0|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20| 3000.0|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|2991.67|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|2518.75|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20| 2175.0|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30| 2850.0|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30| 2225.0|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|1983.33|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30| 1690.0|
| 7654|MAR

In [298]:
windowspec1 = Window.partitionBy('deptno').orderBy(desc('sal'))

avg_sal_dept = emp_df.withColumn(
    'avg_sal', 
    round(avg('sal').over(windowspec1),2)
)

avg_sal_dept.show()

+-----+------+---------+----+----------+----+----+------+-------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|avg_sal|
+-----+------+---------+----+----------+----+----+------+-------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10| 5000.0|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10| 3725.0|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|2916.67|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20| 3000.0|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20| 3000.0|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|2991.67|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|2518.75|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20| 2175.0|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30| 2850.0|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30| 2225.0|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|1983.33|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30| 1690.0|
| 7654|MAR

In [314]:
#부서별 직업별소개 
emp_df.groupBy('deptno', 'job').agg(count('*'), sum('sal')).orderBy('deptno', 'job').show(3)

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
+------+---------+--------+--------+
only showing top 3 rows



In [317]:
emp_df.cube('deptno','job').agg(count("*"), sum('sal')).orderBy('deptno', 'job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  NULL|     NULL|      15|   32225|
|  NULL|  ANALYST|       2|    6000|
|  NULL|    CLERK|       5|    7350|
|  NULL|  MANAGER|       3|    8275|
|  NULL|PRESIDENT|       1|    5000|
|  NULL| SALESMAN|       4|    5600|
|    10|     NULL|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     NULL|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     NULL|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     NULL|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [321]:
emp_df.cube('deptno','job').agg(count("*"), round(avg('sal'),2)).orderBy('deptno', 'job').show()  #최대급여, 최소급여

+------+---------+--------+------------------+
|deptno|      job|count(1)|round(avg(sal), 2)|
+------+---------+--------+------------------+
|  NULL|     NULL|      15|           2148.33|
|  NULL|  ANALYST|       2|            3000.0|
|  NULL|    CLERK|       5|            1470.0|
|  NULL|  MANAGER|       3|           2758.33|
|  NULL|PRESIDENT|       1|            5000.0|
|  NULL| SALESMAN|       4|            1400.0|
|    10|     NULL|       3|           2916.67|
|    10|    CLERK|       1|            1300.0|
|    10|  MANAGER|       1|            2450.0|
|    10|PRESIDENT|       1|            5000.0|
|    20|     NULL|       5|            2175.0|
|    20|  ANALYST|       2|            3000.0|
|    20|    CLERK|       2|             950.0|
|    20|  MANAGER|       1|            2975.0|
|    30|     NULL|       6|           1566.67|
|    30|    CLERK|       1|             950.0|
|    30|  MANAGER|       1|            2850.0|
|    30| SALESMAN|       4|            1400.0|
|    70|     

In [322]:
emp_df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: date (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [324]:
emp_dept_df = emp_df.join(dept_df, emp_df['deptno']==dept_df['deptno'])
emp_dept_df.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|    10|A

In [326]:
join_df =emp_df.join(dept_df, on='deptno', how='inner')

In [328]:
join_df.show()

+------+-----+------+---------+----+----------+----+----+----------+--------+
|deptno|empno| ename|      job| mgr|  hiredate| sal|comm|     dname|     loc|
+------+-----+------+---------+----+----------+----+----+----------+--------+
|    20| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|  RESEARCH|  DALLAS|
|    30| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|     SALES| CHICAGO|
|    30| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|     SALES| CHICAGO|
|    20| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|  RESEARCH|  DALLAS|
|    30| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|     SALES| CHICAGO|
|    30| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|     SALES| CHICAGO|
|    10| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|ACCOUNTING|NEW YORK|
|    20| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|  RESEARCH|  DALLAS|
|    10| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|ACCOUNTING|NEW YORK|
|    30| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|     SA

In [329]:
join_df.select('deptno','empno','ename','job').show()

+------+-----+------+---------+
|deptno|empno| ename|      job|
+------+-----+------+---------+
|    20| 7369| SMITH|    CLERK|
|    30| 7499| ALLEN| SALESMAN|
|    30| 7521|  WARD| SALESMAN|
|    20| 7566| JONES|  MANAGER|
|    30| 7654|MARTIN| SALESMAN|
|    30| 7698| BLAKE|  MANAGER|
|    10| 7782| CLARK|  MANAGER|
|    20| 7788| SCOTT|  ANALYST|
|    10| 7839|  KING|PRESIDENT|
|    30| 7844|TURNER| SALESMAN|
|    20| 7876| ADAMS|    CLERK|
|    30| 7900| JAMES|    CLERK|
|    20| 7902|  FORD|  ANALYST|
|    10| 7934|MILLER|    CLERK|
+------+-----+------+---------+



In [330]:
spark.stop()